In [1]:
"""Create partitions from the data model based on the Parent names

Args:
    data_model (object): dataframe that contains the data model
    partition_path (str): directory to store the partitioned data model as CSVs
"""

import glob
import os
import pandas as pd
import numpy as np
import re
import datetime
import synapseclient as sc

syn = sc.login()

dm_path = '../../EL.data.model.csv'
output_path = "../backups"

partition_path = "../modules"
# data_model = utils.load_and_backup_dm(dm_name, output_path)


UPGRADE AVAILABLE

A more recent version of the Synapse Client (4.2.0) is available. Your version (3.2.0) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 4.2.0 release notes

https://python-docs.synapse.org/news/



Welcome, Nicholas Lee!



In [ ]:
from pathlib import Path

In [78]:
from tqdm import tqdm

In [2]:
dm = pd.read_csv(dm_path)
dm['module'] = dm['module'].fillna('Unknown')
dm.head()

,Description,Valid Values,DependsOn,DependsOn Component,Required,Parent,Validation Rules,Properties,Source,module,multivalue,columnType,Ontology,Notes,UsedIn
Attribute,,,,,,,,,,,,,,,
acquisitionBatchID,"Acquisition batch identifier, provided by the ...",NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,Metabolomics
acquisitionBatchSize,The number of samples,NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,Metabolomics
acquisitionBatchSizeUnit,The unit of measurement for number of samples ...,NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,Metabolomics
acquisitionMode,The specific aspect of a mass spectrometer met...,NaN,NaN,NaN,True,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,https://www.ebi.ac.uk/ols/ontologies/ms/terms?...,A data contributor should be able to write in ...,Metabolomics
acquisitionSoftware,The name of the acquisition software used. Pro...,NaN,NaN,NaN,True,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,http://purl.obolibrary.org/obo/MS_1001455,A data contributor should be able to write in ...,Metabolomics


In [3]:
partition_path = "../../modules"

# create_data_model_partitions(data_model, partition_path)

In [19]:
# Split by Parent
modules = dm["module"].dropna().unique()
modules

array(['ManifestColumn', 'individual', 'experimentalData', 'batch',
       'template', 'biospecimen', 'clinical', 'sequencing',
       'demographics', 'mass spectrometry', 'ADKP'], dtype=object)

In [92]:
import shutil

# clean modules folders
existing_dir_mods = [s.strip('/') for s in glob.glob("**/", root_dir = partition_path)]

for e in list(set(existing_dir_mods) - set(modules)): 
    print('Removing directory: ', e)
    shutil.rmtree(Path(partition_path, e))

In [93]:
# create module directories
for m in list(set(existing_dir_mods) - set(modules)):
    mod_path = Path(partition_path, m)

    print(mod_path)

    mod_path.mkdir(parents = True, exist_ok=True)

In [94]:
# check for existing csv files
existing_csvs = glob.glob('**/*.csv', recursive = True, root_dir = partition_path)
existing_attrs = [Path(e).stem for e in existing_csvs]
existing_df = pd.DataFrame(
    {"csv_paths": existing_csvs, "existing_attrs": existing_attrs}
)
existing_df["module"] = existing_df["csv_paths"].apply(lambda x: str(Path(x).parent))

{'sequencing/libraryPrep.csv': 'libraryPrep',
 'sequencing/readLength.csv': 'readLength',
 'sequencing/libraryPreparationMethod.csv': 'libraryPreparationMethod',
 'sequencing/lambdaDNAconc.csv': 'lambdaDNAconc',
 'sequencing/isStranded.csv': 'isStranded',
 'sequencing/meanCoverage.csv': 'meanCoverage',
 'sequencing/totalReads.csv': 'totalReads',
 'sequencing/gDNAconc.csv': 'gDNAconc',
 'sequencing/runType.csv': 'runType',
 'sequencing/directionalBSseqLibrary.csv': 'directionalBSseqLibrary',
 'sequencing/libraryVersion.csv': 'libraryVersion',
 'sequencing/pcrCycles.csv': 'pcrCycles',
 'sequencing/readStrandOrigin.csv': 'readStrandOrigin',
 'demographics/ethnicity.csv': 'ethnicity',
 'demographics/sex.csv': 'sex',
 'demographics/race.csv': 'race',
 'template/assay_phenotype_human_template.csv': 'assay_phenotype_human_template',
 'template/assay_scRNAseq_template.csv': 'assay_scRNAseq_template',
 'template/individual_human_template.csv': 'individual_human_template',
 'template/individual_

In [110]:
for i in tqdm(range(len(dm)), desc = 'Updating module CSVs'): 
    try: 
        attr = dm.loc[i, 'Attribute']
        module = dm.loc[i, 'module']
        csv_path = Path(module, attr + '.csv')
        full_csv_path = Path(partition_path, csv_path)
        
        checker = existing_df[existing_df['existing_attrs'] == attr]
        
        if all(checker['module'] == module): 
            # print('CSV already exists')
            pass # need to do comparison later
        
        elif all(checker['module'] != module):  # if module is different
            # remove old csv
            print('Removing old CSV file: ', attr)
            Path.unlink(full_csv_path)
            
            # create new csv
            print('Creating new CSV: ', csv_path)
            dm.loc[i, ].to_csv(full_csv_path, index = 'ignore')
        
        else:     
            # create new csv
            dm.loc[i, ].to_csv(full_csv_path, index = 'ignore')
    
    except Exception as e: 
        print(attr, e)
        pass

Updating module CSVs: 100%|██████████| 238/238 [00:00<00:00, 5499.18it/s]

CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already exists
CSV already 